In [ ]:
import numpy as np

from lymph.models import Unilateral
from lymixture import LymphMixtureModel
from lymixture.utils import map_to_simplex
from fixtures import (
    get_graph,
    get_patient_data,
    SIMPLE_SUBSITE,
)

In [ ]:
graph = get_graph(size="medium")
patient_data = get_patient_data()
num_components = 3

mixture = LymphMixtureModel(
    model_cls=Unilateral,
    model_kwargs={"graph_dict": graph},
    num_components=num_components,
)
mixture.load_patient_data(patient_data, split_by=SIMPLE_SUBSITE)

In [ ]:
resp_from_cube = np.random.uniform(size=(len(patient_data), num_components-1))
resp = np.array([map_to_simplex(line) for line in resp_from_cube])

mixture.assign_responsibilities(resp)
mixture.get_responsibilities(patient=0, component=1)

In [ ]:
tmp = np.random.uniform(size=(num_components, len(mixture.subgroups)))
tmp /= tmp.sum(axis=0)
mixture.assign_mixture_coefs(tmp)

In [ ]:
mixture.get_mixture_coefs()